In [2]:
# Load all dependency
%load_ext autoreload
%autoreload 2

!pip install torchviz
!pip install transforms3d

from IPython.display import HTML
from IPython.display import clear_output

import numpy as np

import torch
from torch import nn
from torch.nn import modules
from torchvision import transforms, models
import torchvision.utils as vutils
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from torchviz import make_dot

from datasets.apolloscape import Apolloscape

from utils.common import draw_poses
from utils.common import draw_record
from utils.common import imshow
from utils.common import save_checkpoint
from utils.common import AverageMeter
from utils.common import calc_poses_params, quaternion_angular_error

from models.posenet import PoseNet, PoseNetCriterion
from models.posenet1 import PoseNet1
from models.posenet2 import PoseNet2

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

from PIL import Image

from tqdm import tqdm
import os
import time
from datetime import datetime

%matplotlib inline
plt.ion()

from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import LabelEncoder

from mlxtend.classifier import StackingClassifier
from mlxtend.feature_selection import ColumnSelector


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
APOLLO_PATH = "./drive/My Drive/Colab Notebooks/Final_Project/apolloscape"

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])

# Resize data before using
transform = transforms.Compose([
    transforms.Resize(260),
    transforms.CenterCrop(250),
    transforms.ToTensor(),
    normalize
])

experiment_name = 'zpark_L6'

pretrained = True
stereo = False
shuffle = True

batch_size = 40

train_record = None # 'Record001'
train_dataset = Apolloscape(root=os.path.join(APOLLO_PATH), road="zpark-sample",
                             transform=transform, record=train_record, normalize_poses=True,
                             pose_format='quat', train=True, cache_transform=False, stereo=stereo)

val_record = None # 'Record013'
val_dataset = Apolloscape(root=os.path.join(APOLLO_PATH), road="zpark-sample",
                             transform=transform, record=val_record, normalize_poses=True,
                             pose_format='quat', train=False, cache_transform=False, stereo=stereo)


print(train_dataset)
print(val_dataset)

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=shuffle) # batch_size = 75
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=shuffle) # batch_size = 75

Dataset: Apolloscape
    Road: zpark-sample
    Record: None
    Train: True
    Normalize Poses: True
    Stereo: False
    Length: 2242 of 2242
    Cameras: ['Camera_2', 'Camera_1']
    Records: ['Record001', 'Record002', 'Record003', 'Record004', 'Record006', 'Record007', 'Record008', 'Record009', 'Record010', 'Record011', 'Record012', 'Record013', 'Record014']

Dataset: Apolloscape
    Road: zpark-sample
    Record: None
    Train: False
    Normalize Poses: True
    Stereo: False
    Length: 756 of 756
    Cameras: ['Camera_2', 'Camera_1']
    Records: ['Record001', 'Record002', 'Record003', 'Record004', 'Record006', 'Record007', 'Record008', 'Record009', 'Record010', 'Record011', 'Record012', 'Record013', 'Record014']



In [5]:
device = None
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
print('device = {}'.format(device))

device = cuda


In [6]:
# Create pretrained feature extractor
# feature_extractor = models.resnet18(pretrained=True)
from torch.autograd import Variable

feature_extractor_1 = models.resnet18(pretrained=True)
feature_extractor_2 = models.resnet34(pretrained=True)
feature_extractor_3 = models.resnet101(pretrained=True)
feature_extractor_4 = models.vgg16(pretrained=True)
#feature_extractor_5 = models.alexnet(pretrained=True)
feature_extractor_6 = models.vgg19(pretrained=True)
#feature_extractor_7 = models.resnet50(pretrained=True)
num_features = 2048

# Create model
model1 = PoseNet(feature_extractor_1, num_features=num_features, pretrained=pretrained)
model1 = model1.to(device)

model2 = PoseNet(feature_extractor_2, num_features=num_features, pretrained=pretrained)
model2 = model2.to(device)

model3 = PoseNet(feature_extractor_3, num_features=num_features, pretrained=pretrained)
model3 = model3.to(device)

model4 = PoseNet1(feature_extractor_4, num_features=num_features, pretrained=pretrained)
model4 = model4.to(device)

#model5 = PoseNet2(feature_extractor_5, num_features=num_features, pretrained=pretrained)
#model5 = model5.to(device)

model6 = PoseNet1(feature_extractor_6, num_features=num_features, pretrained=pretrained)
model6 = model6.to(device)

#model7 = PoseNet(feature_extractor_7, num_features=num_features, pretrained=pretrained)
#model7 = model7.to(device)

start_epoch = 0

# Criterion
criterion_1 = PoseNetCriterion(stereo=stereo, learn_beta=True)
criterion_1 = criterion_1.to(device)

criterion_2 = PoseNetCriterion(stereo=stereo, learn_beta=True)
criterion_2 = criterion_2.to(device)

criterion_3 = PoseNetCriterion(stereo=stereo, learn_beta=True)
criterion_3 = criterion_3.to(device)

criterion_4 = PoseNetCriterion(stereo=stereo, learn_beta=True)
criterion_4 = criterion_4.to(device)

#criterion_5 = PoseNetCriterion(stereo=stereo, learn_beta=True)
#criterion_5 = criterion_5.to(device)

criterion_6 = PoseNetCriterion(stereo=stereo, learn_beta=True)
criterion_6 = criterion_6.to(device)

#criterion_7 = PoseNetCriterion(stereo=stereo, learn_beta=True)
#criterion_7 = criterion_7.to(device)
# Add all params for optimization

param_list_1 = [{'params': model1.parameters()}]
if criterion_1.learn_beta:
    param_list_1.append({'params': criterion_1.parameters()})

param_list_2 = [{'params': model2.parameters()}]
if criterion_2.learn_beta:
    param_list_2.append({'params': criterion_2.parameters()})

param_list_3 = [{'params': model3.parameters()}]
if criterion_1.learn_beta:
    param_list_3.append({'params': criterion_3.parameters()})

param_list_4 = [{'params': model4.parameters()}]
if criterion_4.learn_beta:
    param_list_4.append({'params': criterion_4.parameters()})

#param_list_5 = [{'params': model5.parameters()}]
#if criterion_5.learn_beta:
#    param_list_5.append({'params': criterion_5.parameters()})

param_list_6 = [{'params': model6.parameters()}]
if criterion_6.learn_beta:
    param_list_6.append({'params': criterion_6.parameters()})

#param_list_7 = [{'params': model7.parameters()}]
#if criterion_7.learn_beta:
#    param_list_7.append({'params': criterion_7.parameters()})

# Create optimizer
optimizer_1 = optim.Adam(params=param_list_1, lr=1e-5, weight_decay=0.0005)
optimizer_2 = optim.Adam(params=param_list_2, lr=1e-5, weight_decay=0.0005)
optimizer_3 = optim.Adam(params=param_list_3, lr=1e-5, weight_decay=0.0005)
optimizer_4 = optim.Adam(params=param_list_4, lr=1e-5, weight_decay=0.0005)
#optimizer_5 = optim.Adam(params=param_list_5, lr=1e-5, weight_decay=0.0005)
optimizer_6 = optim.Adam(params=param_list_6, lr=1e-5, weight_decay=0.0005)
#optimizer_7 = optim.Adam(params=param_list_7, lr=1e-5, weight_decay=0.0005)

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/checkpoints/resnet18-5c106cde.pth
100%|██████████| 44.7M/44.7M [00:01<00:00, 41.1MB/s]
Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /root/.cache/torch/checkpoints/resnet34-333f7ec4.pth
100%|██████████| 83.3M/83.3M [00:00<00:00, 154MB/s]
Downloading: "https://download.pytorch.org/models/resnet101-5d3b4d8f.pth" to /root/.cache/torch/checkpoints/resnet101-5d3b4d8f.pth
100%|██████████| 170M/170M [00:03<00:00, 55.6MB/s]
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:08<00:00, 63.3MB/s]
Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/checkpoints/vgg19-dcbb9e9d.pth
100%|██████████| 548M/548M [00:26<00:00, 21.9MB/s]


In [7]:
# Restore from checkpoint
# checkpoint_file = '_checkpoints/20180823_085404_zpark_posenet_L1_resnet34p_2048_e2880.pth.tar'
checkpoint_file_1 = './drive/My Drive/Colab Notebooks/Resnet18Model.pth.tar'

if 'checkpoint_file_1' in locals() and checkpoint_file_1 is not None:
    if os.path.isfile(checkpoint_file_1):
        print('Loading from checkpoint: {}'.format(checkpoint_file_1))
        checkpoint_1 = torch.load(checkpoint_file_1, map_location=torch.device('cpu'))
        model1.load_state_dict(checkpoint_1['model_state_dict'])
        optimizer_1.load_state_dict(checkpoint_1['optim_state_dict'])
        start_epoch = checkpoint_1['epoch']
        if 'criterion_state_dict' in checkpoint_1:
            criterion_1.load_state_dict(checkpoint_1['criterion_state_dict'])
            print('Loaded params.')

checkpoint_file_2 = './drive/My Drive/Colab Notebooks/Resnet34Model.pth.tar'

if 'checkpoint_file_2' in locals() and checkpoint_file_2 is not None:
    if os.path.isfile(checkpoint_file_2):
        print('Loading from checkpoint: {}'.format(checkpoint_file_2))
        checkpoint_2 = torch.load(checkpoint_file_2, map_location=torch.device('cpu'))
        model2.load_state_dict(checkpoint_2['model_state_dict'])
        optimizer_2.load_state_dict(checkpoint_2['optim_state_dict'])
        start_epoch = checkpoint_2['epoch']
        if 'criterion_state_dict' in checkpoint_2:
            criterion_2.load_state_dict(checkpoint_2['criterion_state_dict'])
            print('Loaded params.')

checkpoint_file_3 = './drive/My Drive/Colab Notebooks/Resnet101Model.pth.tar'

if 'checkpoint_file_3' in locals() and checkpoint_file_3 is not None:
    if os.path.isfile(checkpoint_file_3):
        print('Loading from checkpoint: {}'.format(checkpoint_file_3))
        checkpoint_3 = torch.load(checkpoint_file_3, map_location=torch.device('cpu'))
        model3.load_state_dict(checkpoint_3['model_state_dict'])
        optimizer_3.load_state_dict(checkpoint_3['optim_state_dict'])
        start_epoch = checkpoint_3['epoch']
        if 'criterion_state_dict' in checkpoint_3:
            criterion_3.load_state_dict(checkpoint_3['criterion_state_dict'])
            print('Loaded params.')

checkpoint_file_4 = './drive/My Drive/Colab Notebooks/Vgg16Model.pth.tar'

if 'checkpoint_file_4' in locals() and checkpoint_file_4 is not None:
    if os.path.isfile(checkpoint_file_4):
        print('Loading from checkpoint: {}'.format(checkpoint_file_4))
        checkpoint_4 = torch.load(checkpoint_file_4, map_location=torch.device('cpu'))
        model4.load_state_dict(checkpoint_4['model_state_dict'])
        optimizer_4.load_state_dict(checkpoint_4['optim_state_dict'])
        start_epoch = checkpoint_4['epoch']
        if 'criterion_state_dict' in checkpoint_4:
            criterion_4.load_state_dict(checkpoint_4['criterion_state_dict'])
            print('Loaded params.')

#checkpoint_file_7 = './drive/My Drive/Colab Notebooks/Resnet50Model.pth.tar'

#if 'checkpoint_file_7' in locals() and checkpoint_file_7 is not None:
#    if os.path.isfile(checkpoint_file_7):
#        print('Loading from checkpoint: {}'.format(checkpoint_file_7))
#        checkpoint_7 = torch.load(checkpoint_file_7, map_location=torch.device('cpu'))
#        model7.load_state_dict(checkpoint_7['model_state_dict'])
#        optimizer_7.load_state_dict(checkpoint_7['optim_state_dict'])
#        start_epoch = checkpoint_7['epoch']
#        if 'criterion_state_dict' in checkpoint_7:
#            criterion_7.load_state_dict(checkpoint_7['criterion_state_dict'])
#            print('Loaded params.')

checkpoint_file_6 = './drive/My Drive/Colab Notebooks/Vgg19Model.pth.tar'

if 'checkpoint_file_6' in locals() and checkpoint_file_6 is not None:
    if os.path.isfile(checkpoint_file_6):
        print('Loading from checkpoint: {}'.format(checkpoint_file_6))
        checkpoint_6 = torch.load(checkpoint_file_6, map_location=torch.device('cpu'))
        model6.load_state_dict(checkpoint_6['model_state_dict'])
        optimizer_6.load_state_dict(checkpoint_6['optim_state_dict'])
        start_epoch = checkpoint_6['epoch']
        if 'criterion_state_dict' in checkpoint_6:
            criterion_6.load_state_dict(checkpoint_6['criterion_state_dict'])
            print('Loaded params.')

Loading from checkpoint: ./drive/My Drive/Colab Notebooks/Resnet18Model.pth.tar
Loaded params.
Loading from checkpoint: ./drive/My Drive/Colab Notebooks/Resnet34Model.pth.tar
Loaded params.
Loading from checkpoint: ./drive/My Drive/Colab Notebooks/Resnet101Model.pth.tar
Loaded params.
Loading from checkpoint: ./drive/My Drive/Colab Notebooks/Vgg16Model.pth.tar
Loaded params.
Loading from checkpoint: ./drive/My Drive/Colab Notebooks/Vgg19Model.pth.tar
Loaded params.


In [0]:
def model_results_pred_gt(model, dataloader, poses_mean, poses_std, stereo=True):
    model.eval()

    gt_poses = np.empty((0, 7))
    pred_poses = np.empty((0, 7))

    for idx, (batch_images, batch_poses) in enumerate(dataloader):
        
        if stereo:
            batch_images = [x.to(device) for x in batch_images]
            batch_poses = [x.to(device) for x in batch_poses]
        else:
            batch_images = batch_images.to(device)
            batch_poses = batch_poses.to(device)


        out = model(batch_images)
        
        loss = criterion_1(out, batch_poses)

        # move data to cpu & numpy
        if stereo:
            batch_poses = [x.detach().cpu().numpy() for x in batch_poses]
            out = [x.detach().cpu().numpy() for x in out]
            gt_poses = np.vstack((gt_poses, *batch_poses))
            pred_poses = np.vstack((pred_poses, *out))
        else:
            bp = batch_poses.detach().cpu().numpy()
            outp = out.detach().cpu().numpy()
            gt_poses = np.vstack((gt_poses, bp))
            pred_poses = np.vstack((pred_poses, outp))


        
    # un-normalize translation
    gt_poses[:, :3] = gt_poses[:, :3] * poses_std + poses_mean
    pred_poses[:, :3] = pred_poses[:, :3] * poses_std + poses_mean
    
    return pred_poses, gt_poses

In [10]:
#Get mean and std from dataset
poses_mean = val_dataset.poses_mean
poses_std = val_dataset.poses_std

print('\n=== Test Validation Dataset ======')
pred_poses_1, gt_poses_1 = model_results_pred_gt(model1, val_dataloader, poses_mean, poses_std, stereo=stereo)
pred_poses_2, gt_poses_2 = model_results_pred_gt(model2, val_dataloader, poses_mean, poses_std, stereo=stereo)
pred_poses_3, gt_poses_3 = model_results_pred_gt(model3, val_dataloader, poses_mean, poses_std, stereo=stereo)
pred_poses_4, gt_poses_4 = model_results_pred_gt(model4, val_dataloader, poses_mean, poses_std, stereo=stereo)
#pred_poses_5, gt_poses_5 = model_results_pred_gt(model5, val_dataloader, poses_mean, poses_std, stereo=stereo)
pred_poses_6, gt_poses_6 = model_results_pred_gt(model6, val_dataloader, poses_mean, poses_std, stereo=stereo)
#pred_poses_7, gt_poses_7 = model_results_pred_gt(model7, val_dataloader, poses_mean, poses_std, stereo=stereo)

pred_poses = (pred_poses_1 + pred_poses_2 + pred_poses_3 + pred_poses_4 + pred_poses_6)/5
gt_poses = (gt_poses_1 + gt_poses_2 + gt_poses_3 + gt_poses_4 + gt_poses_6)/5

#np.savetxt("pred_poses.csv", pred_poses, delimiter=',', comments="")
#np.savetxt("gt_poses.csv", gt_poses, delimiter=',', comments="")

print('gt_poses = {}'.format(gt_poses.shape))
print('pred_poses = {}'.format(pred_poses.shape))
t_loss = np.asarray([np.linalg.norm(p - t) for p, t in zip(pred_poses[:, :3], gt_poses[:, :3])])
q_loss = np.asarray([quaternion_angular_error(p, t) for p, t in zip(pred_poses[:, 3:], gt_poses[:, 3:])])

print('poses_std = {:.3f}'.format(np.linalg.norm(poses_std)))
print('Translation(T) error in meters and Rotation(R) error in degrees:')
print('T: median = {:.3f}, mean = {:.3f}'.format(np.median(t_loss), np.mean(t_loss)))
print('R: median = {:.3f}, mean = {:.3f}'.format(np.median(q_loss), np.mean(q_loss)))

# Save for later visualization
#pred_poses_val = pred_poses
#gt_poses_val = gt_poses


=== Test Validation Dataset ======
gt_poses = (756, 7)
pred_poses = (756, 7)
poses_std = 280.970
Translation(T) error in meters and Rotation(R) error in degrees:
T: median = 9.288, mean = 10.352
R: median = 0.931, mean = 4.151


In [18]:
data1 = np.loadtxt('pred_poses_1')
print(data1)

[[ 5.47409564e+02 -2.22899669e+03  4.02282543e+01 ... -2.87257016e-01
   7.22187877e-01 -5.86950064e-01]
 [ 3.26294691e+02 -2.39396893e+03  4.06166079e+01 ... -7.79828072e-01
  -1.62897632e-03  1.92154981e-02]
 [ 5.34936891e+02 -2.10705136e+03  4.01266676e+01 ... -1.33494392e-01
  -7.63605416e-01  6.13447189e-01]
 ...
 [ 3.52307500e+02 -1.91605973e+03  4.00668908e+01 ... -5.63804507e-01
   5.41498005e-01 -4.25178409e-01]
 [ 3.66331748e+02 -2.61325011e+03  4.02018798e+01 ... -7.28249133e-01
  -2.59955198e-01  1.61541909e-01]
 [ 3.13951524e+02 -2.30156566e+03  4.05283709e+01 ... -7.85009861e-01
  -4.05925512e-03  2.00082511e-02]]


In [19]:
print(pred_poses_1)

[[ 5.47409564e+02 -2.22899669e+03  4.02282543e+01 ... -2.87257016e-01
   7.22187877e-01 -5.86950064e-01]
 [ 3.26294691e+02 -2.39396893e+03  4.06166079e+01 ... -7.79828072e-01
  -1.62897632e-03  1.92154981e-02]
 [ 5.34936891e+02 -2.10705136e+03  4.01266676e+01 ... -1.33494392e-01
  -7.63605416e-01  6.13447189e-01]
 ...
 [ 3.52307500e+02 -1.91605973e+03  4.00668908e+01 ... -5.63804507e-01
   5.41498005e-01 -4.25178409e-01]
 [ 3.66331748e+02 -2.61325011e+03  4.02018798e+01 ... -7.28249133e-01
  -2.59955198e-01  1.61541909e-01]
 [ 3.13951524e+02 -2.30156566e+03  4.05283709e+01 ... -7.85009861e-01
  -4.05925512e-03  2.00082511e-02]]


In [0]:
np.savetxt("pred_poses_1", pred_poses_1)
np.savetxt("pred_poses_2", pred_poses_2)
np.savetxt("pred_poses_3", pred_poses_3)
np.savetxt("pred_poses_4", pred_poses_4)
np.savetxt("pred_poses_6", pred_poses_6)

np.savetxt("gt_poses_1", gt_poses_1)
np.savetxt("gt_poses_2", gt_poses_2)
np.savetxt("gt_poses_3", gt_poses_3)
np.savetxt("gt_poses_4", gt_poses_4)
np.savetxt("gt_poses_6", gt_poses_6)

In [0]:
np.savetxt("pred_poses_1", pred_poses_1)